In [8]:
import os
import nest_asyncio
import asyncio
import logging
from playwright.async_api import async_playwright
nest_asyncio.apply()

logging.basicConfig(
    filename="ui_test_actions.log",
    filemode="a",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

In [9]:
async def test_add_patient(patient_id: str):    
    logging.info(f"Test start: Adding patient {patient_id}")
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        html_path = os.path.abspath("mock_ui.html")
        await page.goto(f"file://{html_path}")

        await page.fill('input[name="patient_id"]', patient_id)
        await page.fill('input[name="age"]', "45")
        await page.fill('input[name="sex"]', "Male")
        await page.fill('input[name="BMI"]', "25.4")
        await page.fill('input[name="diagnosis_code"]', "I99")
        await page.fill('input[name="blood_pressure"]', "122/78")
        await page.fill('input[name="cholesterol_level"]', "5.9")
        await page.check('input[name="sample_taken"]')
        await page.click('button[type="submit"]')

        await page.wait_for_selector(f'tr:has-text("{patient_id}")')
        print(f"✅ Added patient {patient_id}")
        
        logging.info(f"✅ Successfully added patient {patient_id}")
        await browser.close()

In [10]:
async def test_edit_patient(patient_id: str, new_age: str, new_bmi: str):    
    logging.info(f"Test start: Editing patient {patient_id} → age={new_age}, BMI={new_bmi}")
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        html_path = os.path.abspath("mock_ui.html")
        await page.goto(f"file://{html_path}")

        await page.click(f'tr:has-text("{patient_id}") >> text=Edit')
        await page.fill('input[name="age"]', new_age)
        await page.fill('input[name="BMI"]', new_bmi)
        await page.click('button[type="submit"]')
        await page.wait_for_timeout(500)

        updated_row = await page.locator(f'tr:has-text("{patient_id}")').inner_text()
        assert new_age in updated_row and new_bmi in updated_row
        print(f"✅ Edited patient {patient_id}: age={new_age}, BMI={new_bmi}")
        
        logging.info(f"✅ Successfully edited patient {patient_id}")
        await browser.close()

In [11]:
async def test_delete_patient(patient_id: str):    
    logging.info(f"Test start: Deleting patient {patient_id}")
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        html_path = os.path.abspath("mock_ui.html")
        await page.goto(f"file://{html_path}")

        await page.evaluate("window.confirm = () => true")
        await page.click(f'tr:has-text("{patient_id}") >> text=Delete')
        await page.wait_for_timeout(1000)

        content = await page.content()
        assert patient_id not in content
        print(f"✅ Deleted patient {patient_id}")

        
        logging.info(f"✅ Successfully deleted patient {patient_id}")
        await browser.close()

In [12]:
await test_add_patient("TEST999")

✅ Added patient TEST999


In [13]:
await test_edit_patient("TEST999", "52", "27.2")

✅ Edited patient TEST999: age=52, BMI=27.2


In [14]:
await test_delete_patient("TEST999")

✅ Deleted patient TEST999
